# Домашка №1

# Загрузка данных

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
train = pd.read_csv('/kaggle/input/2021-homework1-multiclass-classification/train.csv', sep=';')
test = pd.read_csv('/kaggle/input/2021-homework1-multiclass-classification/test.csv', sep=';')
submission = pd.read_csv('/kaggle/input/2021-homework1-multiclass-classification/sample_submission.csv')

In [ ]:
train.head(5)

In [ ]:
'train: ', train.shape, 'test: ', test.shape

Результат нужно сохранить в **csv файл с двумя колонками**: 

listing_id - берете из датасета

interest_level - проставляете своим алгоритмом

Разделитель **запятая**, при сохранении с помощью result.to_csv() - не забывайте указывать index=None.

In [ ]:
submission.head()

## Анализ данных
## 1. Предобработка и генерация признаков

Для выполнения задания попробуйте придумать признаки на основе имеющихся данных, для работы с текстовыми полями вам может пригодится LabelEncoder/OneHotEncoder, re.sub(),re.findall() 

Ниже представлены имеющиеся у нас признаки. Можно попробовать сгенерировать из них новые. Например, можно сделать признак Стоимость за комнату, признаки на основе значений в features, выделить и закодировать адрес, так же можно попробовать оценить "успешность" менеджера, предположив, что чем больше объявлений у данного manager_id - тем он круче.

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
train.interest_level.value_counts()

In [ ]:
train.interest_level.value_counts(normalize=True)

In [ ]:
mapper={
        'low':0,
        'medium':1,
        'high':2
       }

**Преобразование фичи "Уровень интереса"**

In [ ]:
train['interest_level_num'] = train['interest_level'].apply(lambda x: mapper[x])
train['interest_level_num']

**Строки превращаем обратно в список**

In [ ]:
train.features[24555]

In [ ]:
train['features'] = train['features'].str.replace('[\[\]\']', '').str.split(',')

**Преобразование фичи 'building_id'** 

In [ ]:
k = 1
for i in range(len(train['building_id'])):
    if type(train['building_id'][i]) == str:
        temp = train['building_id'][i]
        train['building_id'] = train['building_id'].replace(temp, k)
        k = k+1
train['building_id']

**Чистим фичи от лишнего мусора. Оставляем только латиницу**

In [ ]:
import re
for i in range(len(train['features'])):
    for j in range(len(train['features'][i])):
        train['features'][i][j] = re.sub("[^A-Z|^a-z]", ' ', train['features'][i][j])

In [ ]:
train['features']

**Инициализируем токеннизатор и лемматизатор**

In [ ]:
w_tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

**Производим токенизацию, т.е. разбиение фич на слова**

In [ ]:
for i in range(len(train['features'])):
        train['features'][i] = list(map(w_tokenizer.tokenize, train['features'][i]))

In [ ]:
train['features']

**Производим лемматизацию, т.е. трансформацию слов к начальной словоформе. Формируем датафрейм фич, с которым будем работать в дальнейшем**

In [ ]:
for i in range(len(train['features'])):
    for j in range(len(train['features'][i])):
        for k in range(len(train['features'][i][j])):
            train['features'][i][j][k] = lemmatizer.lemmatize(train['features'][i][j][k].lower())

In [ ]:
train['features']

In [ ]:
for i in range(len(train['features'])):
    for j in range(len(train['features'][i])):
        train['features'][i][j] = " ".join(train['features'][i][j])

**Разбиение датасета в зависимости от уровня интереса для поиска возможной корреляции**

**High**

In [ ]:
delete_dict = []
for i in range(len(train['interest_level_num'])):
    if train['interest_level_num'].iloc[i] != 2:
        delete_dict.append(i)

In [ ]:
train_high = train.drop(delete_dict)

In [ ]:
features_high_list = []
for i in range(len(train_high["features"])):
    for j in range(len(train_high["features"].iloc[i])):
        features_high_list.append(train_high["features"].iloc[i][j])
#features

In [ ]:
features_high = pd.Series(features_high_list)
features_high_value = features_high.value_counts(normalize=True).head(20)
features_high_value

**Meadle**

In [ ]:
delete_dict = []
for i in range(len(train['interest_level_num'])):
    if train['interest_level_num'].iloc[i] != 1:
        delete_dict.append(i)

In [ ]:
train_middle = train.drop(delete_dict)

In [ ]:
features_middle_list = []
for i in range(len(train_middle["features"])):
    for j in range(len(train_middle["features"].iloc[i])):
        features_middle_list.append(train_middle["features"].iloc[i][j])

In [ ]:
features_middle = pd.Series(features_middle_list)
features_middle_value = features_middle.value_counts(normalize=True).head(20)
features_middle_value

**Low**

In [ ]:
delete_dict = []
for i in range(len(train['interest_level_num'])):
    if train['interest_level_num'].iloc[i] != 0:
        delete_dict.append(i)

In [ ]:
train_low = train.drop(delete_dict)

In [ ]:
features_low_list = []
for i in range(len(train_low["features"])):
    for j in range(len(train_low["features"].iloc[i])):
        features_low_list.append(train_low["features"].iloc[i][j])


In [ ]:
features_low = pd.Series(features_low_list)
features_low_value = features_low.value_counts(normalize=True).head(20)
features_low_value


**Формирование датасета фичей**

In [ ]:
train_features = pd.DataFrame(train['features'], columns = ['features'])
train_features

In [ ]:
# функция для поэлементного сравнения двух массивов (возращает 0 при отсутствии общих слов)
def compare_list(list1, list2):
    for i in range(len(list1)):
        for j in range(len(list2)):
            if list1[i] == list2[j]:
                return 1
    return 0  

**Формируем множество всех фич**

In [ ]:
features = set()
for i in range(len(train)):
    for j in range(len(train["features"][i])):
        features.add(train["features"][i][j])
#features

**Напишем функцию поиска по ключевым словам** 

In [ ]:
# функция для формирования новых фичей(поиск вхождения по ключевым словам)
def find_analogy(list_dict, df, name_column, name_new_column):
    for i in range(len(df[name_column])):
        df.loc[i, name_new_column] = 0
        for j in range(len(df[name_column][i])):
            for k in range(len(list_dict)):
                while df[name_column][i][j].find(list_dict[k]) != -1:
                    df.loc[i, name_new_column] = 1
                    break

**Генерация фич по ключевым словам**

In [ ]:
# по описанию
list_size = ['large', 'huge', 'big', 'queen']
list_repair = ['modern', 'renovated', 'new', 'renovation']
list_photo = ['photo', 'actual']

# по характеру владения
list_private = ['private']
list_resident = ['resident']
list_rent = ['rent', 'rentable']

# по цене
list_low_price = ['low fee', 'no fee', 'underpriced', 'lowrise', 'reduced fee']

# по наличию комнат
list_gameroom = ['playroom', 'gameroom', 'game']
list_parking = ['garage', 'parking']
list_laundry = ['laundry', 'washer']
list_storage = ['closet', 'package', 'storage']
list_parking = ['garage', 'parking']

# по удобству и жизнедеятельности
list_transport = ['subway', 'transport']
list_greens = ['park', 'garden', 'yard']
list_child = ['child']
list_pet = ['pet ok', 'dog allowed', 'cat allowed', 'small dog', 'pet allowed', 'pet welcome', 'pet case by case']
list_fitness = ['fitness', 'gym', 'yoga']

# по услугам
list_tv = ['cable', 'tv']
list_internet = ['wifi', 'internet']
list_heat = ['heat', 'hot water']
list_elevator = ['elevator']

#
list_roof = ['roof']
list_doorman = ['doorman']
list_din_room = ['dining']



In [ ]:
find_analogy(list_size, train_features, 'features', 'big_size')
find_analogy(list_repair, train_features, 'features', 'fresh_repair')
find_analogy(list_photo, train_features, 'features', 'actual_photo')
find_analogy(list_private, train_features, 'features', 'private')
find_analogy(list_resident, train_features, 'features', 'resident')
find_analogy(list_rent, train_features, 'features', 'rent')
find_analogy(list_low_price, train_features, 'features', 'low_price')
find_analogy(list_gameroom, train_features, 'features', 'gameroom')
find_analogy(list_storage, train_features, 'features', 'storageroom')
find_analogy(list_laundry, train_features, 'features', 'laundry')
find_analogy(list_parking, train_features, 'features', 'parking')
find_analogy(list_transport, train_features, 'features', 'transport')
find_analogy(list_greens, train_features, 'features', 'greens')
find_analogy(list_child, train_features, 'features', 'for_child')
find_analogy(list_pet, train_features, 'features', 'pet')
find_analogy(list_fitness, train_features, 'features', 'fitness')
find_analogy(list_tv, train_features, 'features', 'tv')
find_analogy(list_internet, train_features, 'features', 'internet')
find_analogy(list_heat, train_features, 'features', 'heat')
find_analogy(list_elevator, train_features, 'features', 'elevator')
find_analogy(list_roof, train_features, 'features', 'roof')
find_analogy(list_doorman, train_features, 'features', 'doorman')
find_analogy(list_din_room, train_features, 'features', 'dining')

In [ ]:
train_features 

**Добавление еще двух фич**

In [ ]:
train_features['bathrooms'] = train['bathrooms']
train_features['bedrooms'] = train['bedrooms']

**Удаление колонки со списками**

In [ ]:
train_features = train_features.drop(['features'], axis=1)

In [ ]:
train_features

**Инициализируем координаты Центрального парка Нью-Йорка**

In [ ]:
latitude_central_park = 40.4657 
longitude_central_park = 73.5758

**Формирование фичей близости по координатам**

In [ ]:
train_features['proximity_сen_park_lat'] = train['latitude'].apply(lambda x: abs(1 - abs(latitude_central_park - abs(x))))
train_features['proximity_сen_park_lon'] = train['longitude'].apply(lambda x: abs(1 - abs(longitude_central_park - abs(x))))

In [ ]:
train_features

**Нормализуем цену по медианной. Добавляем ее еще как одну фичу**

In [ ]:
train['price'].median()

In [ ]:
train_features['price_normalize'] = train['price'].apply(lambda x: x / train['price'].median())

In [ ]:
train_features

**Формируем фичу количества фичей. Нормализуем относительно максимального показателя**

In [ ]:
train_features['num_features'] = train_features['big_size'] + train_features['fresh_repair'] + train_features['actual_photo'] + train_features['private'] + train_features['resident'] + train_features['rent'] + train_features['low_price'] + train_features['gameroom'] + train_features['storageroom'] + train_features['laundry'] + train_features['parking'] + train_features['transport'] + train_features['greens'] + train_features['for_child'] + train_features['pet'] + train_features['tv'] + train_features['internet'] + train_features['heat'] + train_features['elevator'] +  train_features['roof'] + train_features['doorman'] + train_features['dining']

In [ ]:
train_features['num_features'] = train_features['num_features'] / train_features['num_features'].max()

In [ ]:
train_features

**Добавляем переработанный building_id в датасет фич**

In [ ]:
for i in range(len(train['building_id'])):
    train_features.loc[i, 'building_id'] = train['building_id'].iloc[i]

In [ ]:
train_features

## 2. Тестирование различных алгоритмов

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, average_precision_score
import joblib

In [ ]:
classifiers = {
    'KNeighborsClassifier': KNeighborsClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GaussianNB': GaussianNB(),
    'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis(),
    'XGBClassifier': XGBClassifier(),

}

In [ ]:
target='interest_level'
train=train.set_index('listing_id')
train_target=train[target]

In [ ]:
for name, model in classifiers.items():
    print(name)
    X_train, X_test, y_train, y_test=train_test_split(train_features, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state = 2)
    
    model.fit(X_train, y_train)    
    f1 = f1_score(y_test, model.predict(X_test), average = 'macro')
    
    
    print('На отложенной тестовой выборке: ')
    print('%s: F_мера: %7.5f,' %
    (name, f1))
    print(i)
    print('-'*30)

    
    

## 3. Отбор признаков

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_features, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=2)
X_train
X_new = SelectKBest(chi2, k=26).fit_transform(X_train, y_train)

selector = SelectKBest(chi2, k=26)
selector.fit(X_train, y_train)
cols = selector.get_support(indices=True)

classifier = XGBClassifier(random_state=2)
classifier.fit(X_new, y_train) 
f1 = f1_score(y_test, classifier.predict(X_test.iloc[:,cols]), average = 'macro')
f1

**Датасет до отбора**

In [ ]:
X_train

**Датасет после отбора**

In [ ]:
cols
X_new = X_train.iloc[:,cols]
X_new

## 4. Подбор гиперпараметров

In [ ]:
from sklearn.feature_selection import RFE
classifiers_2 = {
'DecisionTreeClassifier': DecisionTreeClassifier(),
'RandomForestClassifier': RandomForestClassifier(),
'AdaBoostClassifier': AdaBoostClassifier(),
'XGBClassifier': XGBClassifier(random_state = 2),
}

# n_features_to_select - кол-во отбираемых фичей

for name, model in classifiers_2.items():
    rfe = RFE(estimator=model, n_features_to_select = 15)
    X_new2 = rfe.fit_transform(train_features,train['interest_level_num'])
    
    X_train, X_test, y_train, y_test=train_test_split(X_new2, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=2)
    model.fit(X_train, y_train)

    f1_res = f1_score(y_test, model.predict(X_test), average = 'macro')

    print('На отложенной тестовой выборке: ')
    print('%s: F_мера: %7.5f,' %
    (name, f1_res))
    print('-'*30)
    rfe_select_features = []
    
    for col, mask in zip(train_features.columns, rfe.support_):
        if mask:
            rfe_select_features.append(col)

**Отобранные гиперпараметры**

In [ ]:
rfe_select_features

## 5. Стэкинг алгоритмов

In [ ]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
clf1 = DecisionTreeClassifier(random_state=42)
clf2 = RandomForestClassifier(n_estimators=50, random_state=42)
clf3 = XGBClassifier()

eclf = VotingClassifier(
    estimators=[('df', clf1), ('rf', clf2), ('xgb', clf3)],
    voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Decision Tree', 'Random Forest', 'XGB', 'Ensemble']):
    scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


## 6. Добавление номера кластера в качестве признака

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_features, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=2)

**Рассчет показателя возможности кластеризации выборки на n-кластеров**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sil = []
kmax = 4

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
    kmeans = KMeans(n_clusters = k).fit(train_features)
    labels = kmeans.labels_
    sil.append(silhouette_score(train_features, labels, metric = 'euclidean'))
sil

**Кластеризация**

In [ ]:
model_n3 = KMeans(n_clusters=3)
model_n3.fit(train_features)

# predict the clusters on the train dataset
predict_train_3 = model_n3.predict(train_features)
print('\nCLusters on train data',predict_train_3) 

**Добавление кластера как фичи**

In [ ]:
for i in range(len(train_features)):
    train_features.loc[i, 'cluster'] = predict_train_3[i]

In [ ]:
train_features

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_features, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=2)
classifier = XGBClassifier(random_state=2)
classifier.fit(X_train, y_train)
y_predicts = classifier.predict(X_test)
f1_score(y_test, y_predicts, average='macro')

## 7. Итоговое решение и сабмит 

Далее должно приводиться ваше итоговое решение, с лучшей метрикой на лидерборде, которой вам удалось добится. Можете сделать два ноутбука если так удобнее.

Выше произведенные действия не гарантируют хороший результат на лидерборде, для него нужно поэксперементировать самостоятельно ;) 

**! ВАЖНО: ваш итоговый результат должен быть как минимум выше baseline.**

## Baseline

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_features, train['interest_level_num'],
                                                  test_size=0.3, stratify=train_target, 
                                                  random_state=2)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
classifier = XGBClassifier(random_state=2)
classifier.fit(X_train, y_train)

In [ ]:
y_predicts = classifier.predict(X_test)
f1_score(y_test, y_predicts, average='macro')

## Подготовка submission

В плане предобработки\генерации признаков делаем все тоже самое. Потом предсказываем моделью значения.

In [ ]:
test['features'] = test['features'].str.replace('[\[\]\']', '').str.split(',')

In [ ]:
k = 1
for i in range(len(test['building_id'])):
    if type(test['building_id'][i]) == str:
        temp = test['building_id'][i]
        test['building_id'] = test['building_id'].replace(temp, k)
        k = k+1
test['building_id']

In [ ]:
for i in range(len(test['features'])):
    for j in range(len(test['features'][i])):
        test['features'][i][j] = re.sub("[^A-Z|^a-z]", ' ', test['features'][i][j])

In [ ]:
for i in range(len(test['features'])):
        test['features'][i] = list(map(w_tokenizer.tokenize, test['features'][i]))

In [ ]:
for i in range(len(test['features'])):
    for j in range(len(test['features'][i])):
        for k in range(len(test['features'][i][j])):
            test['features'][i][j][k] = lemmatizer.lemmatize(test['features'][i][j][k].lower())

In [ ]:
for i in range(len(test['features'])):
    for j in range(len(test['features'][i])):
        test['features'][i][j] = " ".join(test['features'][i][j])

In [ ]:
test_features = pd.DataFrame(test['features'], columns = ['features'])
test_features

In [ ]:
find_analogy(list_size, test_features, 'features', 'big_size')
find_analogy(list_repair, test_features, 'features', 'fresh_repair')
find_analogy(list_photo, test_features, 'features', 'actual_photo')
find_analogy(list_private, test_features, 'features', 'private')
find_analogy(list_resident, test_features, 'features', 'resident')
find_analogy(list_rent, test_features, 'features', 'rent')
find_analogy(list_low_price, test_features, 'features', 'low_price')
find_analogy(list_gameroom, test_features, 'features', 'gameroom')
find_analogy(list_storage, test_features, 'features', 'storageroom')
find_analogy(list_laundry, test_features, 'features', 'laundry')
find_analogy(list_parking, test_features, 'features', 'parking')
find_analogy(list_transport, test_features, 'features', 'transport')
find_analogy(list_greens, test_features, 'features', 'greens')
find_analogy(list_child, test_features, 'features', 'for_child')
find_analogy(list_pet, test_features, 'features', 'pet')
find_analogy(list_fitness, test_features, 'features', 'fitness')
find_analogy(list_tv, test_features, 'features', 'tv')
find_analogy(list_internet, test_features, 'features', 'internet')
find_analogy(list_heat, test_features, 'features', 'heat')
find_analogy(list_elevator, test_features, 'features', 'elevator')
find_analogy(list_roof, test_features, 'features', 'roof')
find_analogy(list_doorman, test_features, 'features', 'doorman')
find_analogy(list_din_room, test_features, 'features', 'dining')

In [ ]:
test_features['bathrooms'] = test['bathrooms']
test_features['bedrooms'] = test['bedrooms']

In [ ]:
test_features = test_features.drop(['features'], axis=1)

In [ ]:
test_features['proximity_сen_park_lat'] = test['latitude'].apply(lambda x: abs(1 - abs(latitude_central_park - abs(x))))
test_features['proximity_сen_park_lon'] = test['longitude'].apply(lambda x: abs(1 - abs(longitude_central_park - abs(x))))

In [ ]:
test_features['price_normalize'] = test['price'].apply(lambda x: x / test['price'].median())

In [ ]:
test_features['num_features'] = test_features['big_size'] + test_features['fresh_repair'] + test_features['actual_photo'] + test_features['private'] + test_features['resident'] + test_features['rent'] + test_features['low_price'] + test_features['gameroom'] + test_features['storageroom'] + test_features['laundry'] + test_features['parking'] + test_features['transport'] + test_features['greens'] + test_features['for_child'] + test_features['pet'] + test_features['tv'] + test_features['internet'] + test_features['heat'] + test_features['elevator'] + test_features['roof'] + test_features['doorman'] + test_features['dining'] 

In [ ]:
test_features['num_features'] = test_features['num_features'] / test_features['num_features'].max()

In [ ]:
for i in range(len(test['building_id'])):
    test_features.loc[i, 'building_id'] = test['building_id'].iloc[i]

**Предсказание тестовской выборки**

In [ ]:
target='interest_level'

In [ ]:
test_features[rfe_select_features]

In [ ]:
test[target] = model.predict(test_features[rfe_select_features])

In [ ]:

test[target]

In [ ]:
test[['listing_id', target]].to_csv('sub_baseline7.csv', index=None)